In [ ]:
# import os
# os.environ["THEANO_FLAGS"] = "compiler=mingw32"

import pymc3 as pm
import numpy as np

with pm.Model() as model:
    I = pm.Categorical('I', [0.8, 0.2])  # P(I)
    R = pm.Categorical('R', [0.4, 0.6])  # P(R)
    M = pm.Categorical('M', [
        [0, 1],  # P(M | I = +i, R = +r)
        [1, 0],  # P(M | I = +i, R = -r)
        [0, 1],  # P(M | I = -i, R = +r)
        [0.7, 0.3]  # P(M | I = -i, R = -r)
    ], shape=4)

    p_IRM = pm.Deterministic('p_IRM', pm.math.switch(
        pm.math.eq(I, 0),
        pm.math.switch(
            pm.math.eq(R, 0),
            pm.math.switch(
                pm.math.eq(M, 0),
                0,
                pm.math.switch(
                    pm.math.eq(M, 1),
                    0,
                    0
                )
            ),
            pm.math.switch(
                pm.math.eq(M, 0),
                0,
                pm.math.switch(
                    pm.math.eq(M, 1),
                    1,
                    0
                )
            )
        ),
        pm.math.switch(
            pm.math.eq(R, 0),
            pm.math.switch(
                pm.math.eq(M, 0),
                0,
                pm.math.switch(
                    pm.math.eq(M, 1),
                    1,
                    0
                )
            ),
            pm.math.switch(
                pm.math.eq(M, 0),
                0.7,
                pm.math.switch(
                    pm.math.eq(M, 1),
                    0.3,
                    0
                )
            )
        )
    ))

    observed_values = {'R': 1}

    trace = pm.sample(10000, tune=2000, cores=1)

    p_irm = pm.summary(trace, var_names=['p_IRM'])
    print("Table with I R M values and P(I,R,M):")
    print(p_irm,type(p_irm))

    p_m = np.mean(trace['M'])
    print("Marginal probability P(+m) that you have the mumps:", p_m)

    p_r_given_m = np.mean(trace['R'][trace['M'][:, 0] == 1])
    print("Probability P(+r | +m) that your roommate has the mumps given that you have the mumps:", p_r_given_m)


Table with I R M values and P(I,R,M):
             mean     sd      hdi_3%   hdi_97%  mcse_mean  mcse_sd  ess_mean  ess_sd  ess_bulk  ess_tail  r_hat
I[0]      0.8323  0.3748    0.0000    1.0000     0.0054   0.0039    7847.0  7847.0    7877.0    5435.0    1.0
R[0]      0.4079  0.4912    0.0000    1.0000     0.0063   0.0044    8060.0  8060.0    8104.0    5389.0    1.0
M[0,0]    0.3844  0.4864    0.0000    1.0000     0.0043   0.0030    8022.0  8006.0    8103.0    5527.0    1.0
M[1,0]    0.6156  0.4864    0.0000    1.0000     0.0043   0.0030    8022.0  8006.0     798.0    5527.0    1.0
M[2,0]    0.6155  0.4863    0.0000    1.0000     0.0043   0.0030    8019.0  7997.0    8025.0    5435.0    1.0
M[3,0]    0.2322  0.4227    0.0000    1.0000     0.0057   0.0040    7931.0  7931.0    7918.0    5343.0    1.0
p_IRM[0]  0.2562  0.4364    0.0000    1.0000     0.0054   0.0038    7852.0  7852.0    7827.0    5389.0    1.0
p_IRM[1]  0.0470  0.2112    0.0000    0.6475     0.0025   0.0018    7981.0  7981.0    7943.0    5527.0    1.0
p_IRM[2]  0.3462  0.4755    0.0000    1.0000     0.0063   0.0044    8080.0  8080.0    8123.0    5435.0    1.0
p_IRM[3]   0.3506  0.4779    0.0000    1.0000     0.0063   0.0044    8062.0  8062.0    8099.0    5359.0    1.0

Marginal probability P(+m) that you have the mumps: 0.5

Probability P(+r | +m) that your roommate has the mumps given that you have the mumps: 0.6156
